# GOV.UK structural network adjacency
We work through a tutorial to learn about key network metrics and statistics based on this [blog post](https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python#fnref:import). 

We will be using the [GOV.UK structural network adjacency list](https://ckan.publishing.service.gov.uk/dataset/gov-uk-structural-network-adjacency-list?utm_source=ckan) data for this tutorial. This shows the connections (edges) between source and sink pages (nodes).

This tutorial will take us through the basics of network visualisations and analysis using NetworkX and Python visualisation tools. The data itself doesn't matter so much, we are learning the principles of graph theory and `networkx` functionality.

For further ideas see the [list of algorithms](https://networkx.github.io/documentation/stable/reference/algorithms/index.html) available for use with networkx.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
import community
import networkx as nx
import os
import pandas as pd

from holoviews.element.graphs import layout_nodes
import holoviews as hv
import holoviews.operation.datashader as hd

# Enable multiline outputs in this notebook
InteractiveShell.ast_node_interactivity = "all"

# Enable interactivity for plots
hv.extension("bokeh")

## Reading the data
Make sure to [download the data]((https://ckan.publishing.service.gov.uk/dataset/gov-uk-structural-network-adjacency-list?utm_source=ckan)), and place it in the `data/processed_network` folder at the parent-level of this repository.

Rather than use the method of reading in the data used in the Quaker tutorial, we will opt for an approach using pandas.

In [ ]:
# Define a relative path to the data
input_dir = os.path.join("..", "..", "data", "processed_network")
data_path = os.path.join(input_dir, "structural_network_adjacency_list_20190301.csv")
data_path

In [ ]:
# Read in the data
df_edges = pd.read_csv(data_path)
df_edges

We now have the edges loaded in, but need a unique set of nodes for our network. Just in case there are some nodes listed in `source_base_path` but not `sink_base_path`, we need to take a unique set across both columns. 

This [SO answer](https://stackoverflow.com/a/26977495) will help us get an array of unique nodes. We will then create a pandas DataFrame, and arbitrarily set the index as `node_id`. Note we have sorted the array of unique nodes for ease here so that the main GOV.UK page slug (`/`) has node ID `0`. 

In [ ]:
# Get an array of unique nodes
unique_nodes = pd.unique(df_edges[["source_base_path", "sink_base_path"]].values.ravel("K"))

# Sort the nodes alphabetically, and artifically create a 'node_id' column using the index
df_nodes = pd.DataFrame(sorted(unique_nodes), columns=["nodes"])
df_nodes.index.name = "node_id"
df_nodes

For completeness, let's label the node IDs for both source and sink pages in `df_edges`.

Here, we merge `df_nodes` to `df_edges` for both `source_base_path` and `sink_base_path` columns. The chained `reset_index` and `rename` functions on `df_nodes` ensure we don't create duplicate source/sink slug columns during the merge, and also create unique node ID column names for the source/sink slug columns.

In [ ]:
# Add the node IDs back in for each of the sources and sinks
for col in df_edges.columns:
    if "_base_path" in col:
        df_edges = df_edges.merge(df_nodes.reset_index().rename(columns={"nodes": col, "node_id": col+"_id"}),
                                  on=col,
                                  how="left", 
                                  validate="m:1")
df_edges